In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
import seaborn as sns


class Embeddings(nn.Module):
    """嵌入层"""

    def __init__(self, d_model, vocab):
        super().__init__()
        self.lut = nn.Embedding(vocab, d_model)  # 定义嵌入层
        self.d_model = d_model  # 定义嵌入层维度

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)  # 嵌入并缩放

In [ ]:
class PositionalEncoding(nn.Module):
    """位置编码"""

    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)  # 定义随机失活
        pe = torch.zeros(max_len, d_model)  # 初始化位置编码张量
        position = torch.arange(0, max_len).unsqueeze(1)  # 位置序号张量
        # 1/10000^(2i/d_model) = e^(-ln(10000) * 2i/d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))  # 频率
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数位置的位置编码
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数位置的位置编码
        pe = pe.unsqueeze(0)  # 在第0维增加一个维度
        self.register_buffer(
            "pe", pe
        )  # 将位置编码张量pe注册为buffer，使其称为模型的一部分，模型保存时缓冲区中内容也会一起被保存起来

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].detach()  # 将位置编码张量与输入张量相加
        return self.dropout(x)

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    """计算缩放点积注意力"""
    d_k = query.size(-1)  # 维度
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)  # 注意力分数
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)  # 替换掩码为0的元素为-1e9，后续经softmax计算后该值会趋近于0
    p_attn = F.softmax(scores, dim=-1)  # 注意力权重
    if dropout is not None:
        p_attn = dropout(p_attn)  # 随机失活
    return torch.matmul(p_attn, value), p_attn

In [ ]:
def clones(module, N):
    """克隆N个相同的模块"""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])


class MultiHeadedAttention(nn.Module):
    """多头注意力"""

    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0  # 维度必须能被h整除
        # 假设d_v等于d_k
        self.d_k = d_model // h  # key的维度
        self.h = h  # 注意力头的数量
        self.linears = clones(nn.Linear(d_model, d_model), 4)  # 创建4个线性层，前三个用于计算注意力，最后一个用于输出
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # 所有注意力头使用相同的掩码
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)  # 批数量
        # 先对输入进行线性变换，再将query,key,value的维度从[nbatches, seq_len, d_model]转换为[nbatches, h, seq_len, d_k]
        query, key, value = [
            l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2) for l, x in zip(self.linears, (query, key, value))
        ]
        # 批量计算注意力
        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)
        # 转换维度[nbatches, h, seq_len, d_k] -> [nbatches, seq_len, h, d_k]，再使内存连续，再合并多个注意力头的结果
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        # 使用最后一个线性层进行线性变换
        return self.linears[-1](x)

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """逐位置前馈网络"""

    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [ ]:
class LayerNorm(nn.Module):
    """层归一化"""

    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection(nn.Module):
    """子层连接结构"""

    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """这里区别于论文LayerNorm(x+Sublayer(x))(Post-LN)，先进行了层归一化(Pre-LN)"""
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer(nn.Module):
    """编码器层"""

    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.self_attn = self_attn  # 多头自注意力层
        self.feed_forward = feed_forward  # 前馈网络层
        self.sublayer = clones(SublayerConnection(size, dropout), 2)  # 克隆2个子层连接，返回一个列表
        self.size = size  # 维度大小

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))  # 子层连接-多头自注意力层，通过匿名函数传参
        return self.sublayer[1](x, self.feed_forward)  # 子层连接-前馈网络层

In [ ]:
class Encoder(nn.Module):
    """编码器"""

    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)  # 克隆N个编码器层
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        """每层依次传入输入和掩码"""
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [ ]:
def subsequent_mask(size):
    """创建掩码张量"""
    attn_shape = (1, size, size)  # 注意力张量形状
    subsequent_mask = np.tril(np.ones(attn_shape))  # 创建下三角矩阵
    return torch.from_numpy(subsequent_mask)

In [ ]:
class DecoderLayer(nn.Module):
    """解码器层"""

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.self_attn = self_attn  # 多头自注意力层
        self.src_attn = src_attn  # 多头注意力层，使用编码器输出作为key,value
        self.feed_forward = feed_forward  # 前馈网络层
        self.sublayer = clones(SublayerConnection(size, dropout), 3)  # 克隆3个子层连接，返回一个列表

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory  # 编码器输出
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))  # 子层连接-多头自注意力层
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))  # 子层连接-多头注意力层
        return self.sublayer[2](x, self.feed_forward)  # 子层连接-前馈网络层

In [ ]:
class Decoder(nn.Module):
    """解码器"""

    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)  # 克隆N个解码器层
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        """每层依次传入输入、编码器输出、源掩码、目标掩码"""
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class Generator(nn.Module):
    """输出层"""

    def __init__(self, d_model, vocab):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)  # log_softmax: 对每个元素做softmax，再取对数，输出一个对数概率分布

In [ ]:
"""测试解码器"""


def make_decoder(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    decoder = Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N)
    return decoder


d_model = 512  # 词向量维度
src_vocab = 10000  # 词表大小
tgt_vocab = 10000  # 词表大小
dropout = 0.1

# 输入部分
position = PositionalEncoding(d_model, dropout)
tgt_embed = nn.Sequential(Embeddings(d_model, tgt_vocab), copy.deepcopy(position))
# 编码器
decoder = make_decoder(src_vocab, tgt_vocab, d_model=d_model, dropout=dropout)

memory = torch.rand(2, 8, 512)  # 编码器输出
src_mask = torch.ones(2, 8).unsqueeze(-2)  # 编码器掩码
x = torch.randint(0, 1000, (2, 8))  # 解码器输入
tgt_mask = subsequent_mask(x.size(1)).unsqueeze(-2)  # 目标掩码

output = decoder(tgt_embed(x), memory, src_mask, tgt_mask)
g = Generator(d_model, tgt_vocab)
out = g(output)  # 解码器输出
out